# A Short Introduction to PySAGA
This is a short introduction to *PySAGA* and its usage including a few simple examples. The *PySAGA* package has been introduced with *SAGA 9.2.0*, which therefore is the minimum requirement for running this script.
### Using PySAGA in *NIX-Like Environments
*Linux*, *FreeBSD* and similar operating systems usually will have installed *SAGA* together with the *PySAGA* package after installation with your distribution's package manager. Alternatively you can build *SAGA* from the source code quite easily by yourself following the instructions from the [Wiki](https://sourceforge.net/p/saga-gis/wiki/Compiling%20SAGA%20on%20Linux/).
### Using PySAGA in Windows Environments
The Windows installation of *SAGA* is portable, i.e. you only need to unzip the binary files and can immediately run the programs, *saga_gui.exe* for starting the graphical user interface or *saga_cmd.exe* to run commands from a terminal or a batch script. To import *PySAGA* in your Python scripts you only need to add the path to your *SAGA* installation to the __PYTHONPATH__ environment variable. You can also do this programmatically within your Python script adding the following command line before trying to import anything from *PySAGA* (adjust the path accordingly):

In [ ]:
import sys; sys.path.insert(1, 'C:/saga-9.3.0_x64')

## Preparations
### Import SAGA API from PySAGA
The *saga_api* module is *PySAGA*'s core component connecting the Python world with *SAGA*. Through *saga_api* almost the complete *SAGA-API* functionality can be accessed, data structures, tools, and many, many helpful functions. So let's import it in the very first place. This might take a few seconds, because it loads not only the *SAGA API* itself but also the tool libraries (and some of these have several dependencies to be loaded too!).

In [ ]:
from PySAGA import saga_api

To check that the import was successful we want to print the loaded tool libraries. To do so we request *SAGA API*'s tool library manager, which provides a summary function. The summary function will return a *saga_api.CSG_String* object, which is not directly compatible with Python strings, but has a suitable conversion function called *c_str()*. The flag *SG_SUMMARY_FMT_FLAT* tells the summary function not give a HTML formatted result string.

In [ ]:
print(saga_api.SG_Get_Tool_Library_Manager().Get_Summary(saga_api.SG_SUMMARY_FMT_FLAT).c_str())

### Get Some Test Data
To show exemplary applications of SAGA functions and tools we need to have some test data. For convenience we want store input as well as resulting data in a separate working directory, so let's create it.

In [ ]:
import os
WorkDir = os.getcwd() + '/test'
if not os.path.exists(WorkDir):
    os.makedirs(WorkDir)
os.chdir(WorkDir)

If not already done we want to download a demo data set from *SAGA*'s [files](https://sourceforge.net/projects/saga-gis/files/SAGA%20-%20Demo%20Data/Demo%20Data%20for%20SAGA/) section hosted at [SourceForge](https://sourceforge.net/projects/saga-gis/). This is a Digital Elevation Model (DEM) of the Mt.St.Helens volcano created from *Shuttle Radar Topograpy Mission* (SRTM) data as provided by the [CGIAR](https://srtm.csi.cgiar.org/).

In [ ]:
File = 'DGM_30m_Mt.St.Helens_SRTM.sgrd'
if not os.path.exists(File):
    from PySAGA.data import helper
    File = helper.Get_File(File='DGM_30m_Mt.St.Helens_SRTM.zip', Local_Dir=WorkDir, Remote_Dir='https://sourceforge.net/projects/saga-gis/files/SAGA%20-%20Demo%20Data/Demo%20Data%20for%20SAGA/')

    import zipfile; zf = zipfile.ZipFile(File, 'r')
    zf.extractall(WorkDir)
    zf.close()

Now we can load the raster DEM using *SAGA API*'s *CSG_Grid()* constructor, which accepts a file name of a raster to be loaded as optional argument. The check for a successful loading of the raster is here done with the *is_Valid()* function of the *CSG_Grid* class.

In [ ]:
dem = saga_api.CSG_Grid(File)
if not dem.is_Valid():
    print('failed to load ' + File)
else:
    print('succcessfully loaded ' + File)

If the file has been loaded successfully into a *saga_api.CSG_Grid* object, we can try to display it with a simple *Plot_Grid()* function provided by *PySAGA*. The plot function uses the [*matplotlib*](https://matplotlib.org/) package and will only work, if it is installed in your *Python* environment.

In [ ]:
from PySAGA import plot

plot.Plot_Grid(dem)

## Running Tools
Assuming that the previous steps were executed successfully, we want to continue with some data analysis.
### Geomorphometric Analysis
We start with a classic in geomorphometric analysis, the derivation of slope angles! The *SAGA* tool doing this job is named *'Slope, Aspect, Curvatures'* and belongs to the tool library *'Morphometry'* of the *'Terrain Analysis'* category. The library's internal name is *ta_morphometry*. First we will import this library from the *PySAGA.tools*. Then we need to create a *SAGA* raster object with *saga_api.CSG_Grid()* that will receive the resulting slope values. As the tool's name implies we can also derive aspect and different flavours of curvature, so let's also get the general curvature. Of course we need to specify an elevation data set, which will be our SRTM DEM for Mt.St.Helens. If the tool execution succeeds we will store the results in our working directory.

In [ ]:
from PySAGA.tools import ta_morphometry

slope = saga_api.CSG_Grid(); curvature = saga_api.CSG_Grid()

if ta_morphometry.Run_Slope_Aspect_Curvature(ELEVATION=dem, SLOPE=slope, UNIT_SLOPE='degree', C_GENE=curvature):
     # save results to file...
    slope.Save('slope.sg-grd-z'); curvature.Save('curvature.sg-grd-z')

    # ...and plot the results!
    from PySAGA import plot; plot.Plot_Grid; plot.Plot_Grid(slope); plot.Plot_Grid(curvature)

Data objects can be used in subsequent calls of *SAGA* tools again to receive new values. We demonstrate this here with the curvature raster object. This time we want to get the tangential curvature.

In [ ]:
if ta_morphometry.Run_Slope_Aspect_Curvature(ELEVATION=dem, C_TANG=curvature):
    curvature.Save('curvature_tangential.sg-grd-z')
    plot.Plot_Grid(curvature)

The tool *'TPI Based Landform Classification'* is using the *'Topograpic Position Index'* to define a landform category for each raster cell. If your IDE supports Intellisense and *SAGA* has been known by the Python environment before loading it (=> PYTHONPATH) you might be able to see the tool's help which will give you more details about the tool. Otherwise you might explore *SAGA GUI* to find the full tool description.

In [ ]:
landforms = saga_api.CSG_Grid()
if ta_morphometry.Run_TPI_Based_Landform_Classification(DEM=dem, LANDFORMS=landforms, RADIUS_A='0; 100', RADIUS_B='100; 1000'):
    landforms.Save('landforms.sg-grd-z')
    plot.Plot_Grid(landforms)

To obtain a smoother, more generalized landform classification we can apply a majority filter which you can find in the *grid_filter* tool box. Notice that this tool will modify the input raster in-place if you do not provide a target raster with the *RESULT* argument.

In [ ]:

from PySAGA.tools import grid_filter

if grid_filter.Run_MajorityMinority_Filter(INPUT=landforms, KERNEL_RADIUS=3):
    landforms.Save('landforms_filtered.sg-grd-z')
    plot.Plot_Grid(landforms)

Landform classifications are typically distributed as vector data. We can derive polygons for each class with the *'Vectorizing Grid Classes'* tool.

In [ ]:
from PySAGA.tools import shapes_grid

polygons = saga_api.CSG_Shapes()
shapes_grid.Run_Vectorizing_Grid_Classes(GRID=landforms, POLYGONS=polygons)

plot.Plot_Shapes(polygons)

In contrast to the previously used tools, the following tool named *'Hypsometry'* will create a result data set of type *saga_api.CSG_Table*. On successful execution we store the table data to file, plot its content using the *PySAGA.plot* function *Plot_Table* and print it using the *PySAGA.helper* function *Print_Table*.

In [ ]:
table = saga_api.CSG_Table()

if ta_morphometry.Run_Hypsometry(ELEVATION=dem, TABLE=table):
    table.Save('hypsometry.txt')

    from PySAGA import plot
    plot.Plot_Table(table, yFields=[1], xField=0)

    from PySAGA import helper
    helper.Print_Table(table)

### Hydrologic Analysis

The relief of the Earth's surface has a strong control on where the water flows and this is what we want to assess now. Typically hydrology related terrain analysis needs spurious pits removed from the original DEM to create a hydrologically sound DEM. We will use the *ta_preprocessor* tool *'Sink Removal'* and compare the preprocessed DEM with the original by calculating their cellwise differences in elevation using the *grid_calculus* tool *'Grid Calculator'*. The grid calculator expects an input grid list, which is simply supplied to it as Python list. Before storing the result we set the no-data value to zero, so that only cells that differ will be shown e.g. visualizing it in *SAGA GUI*.

In [ ]:
from PySAGA.tools import ta_preprocessor

dem_nosinks = saga_api.CSG_Grid()

ta_preprocessor.Run_Sink_Removal(DEM=dem, DEM_PREPROC=dem_nosinks)

# detect the filled sinks
from PySAGA.tools import grid_calculus

sinks = saga_api.CSG_Grid()
grid_calculus.Run_Grid_Calculator(GRIDS=[dem, dem_nosinks], RESULT=sinks, FORMULA='g2 - g1')
sinks.Set_NoData_Value(0.)
sinks.Save('closed_depressions.sg-grd-z')

Once preprocessing is done we can continue deriving the flow accumulation itself. It comes from the *ta_hydrology* library.

In [ ]:
# calculate the flow accumulation using the depressionless DEM
from PySAGA.tools import ta_hydrology

flow_acc = saga_api.CSG_Grid()
ta_hydrology.Run_Flow_Accumulation_TopDown(ELEVATION=dem_nosinks, FLOW=flow_acc, METHOD='Multiple Triangular Flow Directon')
flow_acc.Save('flow_acc.sg-grd-z')
plot.Plot_Grid(flow_acc)

A very popular terrain parameter, the *Topographic Wetness Index* or *TWI*, is calculated from *specific catchment area* (*SCA*) and *slope gradient*. *SCA* is a normalized form of the flow accumulation (aka *total catchment area* or *TCA*). Slope gradient we calculated already but unluckily the *TWI* tool expects slope values in radians (not degree). So let's prepare both input data correctly before we run the *TWI* tool.

In [ ]:
sca = saga_api.CSG_Grid(); twi = saga_api.CSG_Grid()

ta_morphometry.Run_Slope_Aspect_Curvature(ELEVATION=dem, SLOPE=slope, UNIT_SLOPE='radians')
ta_hydrology.Run_Flow_Width_and_Specific_Catchment_Area(DEM=dem_nosinks, TCA=flow_acc, SCA=sca)
ta_hydrology.Run_Topographic_Wetness_Index(SLOPE=slope, AREA=sca, TWI=twi)

twi.Save('twi.sg-grd-z')
plot.Plot_Grid(twi)

### Deriving Contour Lines
In the final example we derive some contour lines from the elevation raster. Target data object type is here of type *saga_api.CSG_Shapes*.

In [ ]:
from PySAGA.tools import shapes_grid

contour = saga_api.CSG_Shapes()

shapes_grid.Run_Contour_Lines_from_Grid(GRID=dem, CONTOUR=contour, INTERVALS='from list', ZLIST='800, 1000, 1200, 1500, 2000')

contour.Save('contour.shp')
plot.Plot_Grid(dem, False) # don't show/finish the plot before the contour lines have been added
plot.Plot_Shapes(contour)

That's it so far!